<a href="https://colab.research.google.com/github/rubenguerrerocano/practica_ia/blob/main/Model_News_to_Summarize_Audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%env CUDA_LAUNCH_BLOCKING=1


env: CUDA_LAUNCH_BLOCKING=1


In [2]:
!pip install transformers
!pip install ftfy

In [3]:
!cd /content
!rm -rf sample_data ChatTTS
!git clone https://github.com/2noise/ChatTTS.git

Cloning into 'ChatTTS'...
remote: Enumerating objects: 2707, done.
remote: Counting objects: 100% (801/801), done.
remote: Compressing objects: 100% (334/334), done.
remote: Total 2707 (delta 565), reused 467 (delta 467), pack-reused 1906 (from 2)
Receiving objects: 100% (2707/2707), 8.01 MiB | 15.25 MiB/s, done.
Resolving deltas: 100% (1634/1634), done.


In [4]:
!pip install -r /content/ChatTTS/requirements.txt
!ldconfig /usr/lib64-nvidia

/sbin/ldconfig.real: /usr/local/lib/libtbb.so.12 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libhwloc.so.15 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libumf.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_0.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind_2_5.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbmalloc_proxy.so.2 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_opencl.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtcm_debug.so.1 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libtbbbind.so.3 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_adapter_level_zero.so.0 is not a symbolic link

/sbin/ldconfig.real: /usr/local/lib/libur_loader.so.0 is

In [5]:
import torch
import torchaudio

torch.random.manual_seed(0)
device = "cuda" if torch.cuda.is_available() else "cpu"

torch._dynamo.config.cache_size_limit = 64
torch._dynamo.config.suppress_errors = True
torch.set_float32_matmul_precision("high")

from ChatTTS import ChatTTS
from ChatTTS.tools.logger import get_logger
from ChatTTS.tools.normalizer import normalizer_en_nemo_text, normalizer_zh_tn
from IPython.display import Audio

In [6]:
logger = get_logger("ChatTTS", format_root=True)
chat = ChatTTS.Chat(logger)

# try to load normalizer
try:
    chat.normalizer.register("en", normalizer_en_nemo_text())
except ValueError as e:
    logger.error(e)
except:
    logger.warning("Package nemo_text_processing not found!")
    logger.warning(
        "Run: conda install -c conda-forge pynini=2.1.5 && pip install nemo_text_processing",
    )
try:
    chat.normalizer.register("zh", normalizer_zh_tn())
except ValueError as e:
    logger.error(e)
except:
    logger.warning("Package WeTextProcessing not found!")
    logger.warning(
        "Run: conda install -c conda-forge pynini=2.1.5 && pip install WeTextProcessing",
    )

 NeMo-text-processing :: INFO     :: Creating ClassifyFst grammars.
[+0000 20250301 19:39:44] [INFO] NeMo-text-processing | tokenize_and_classify | Creating ClassifyFst grammars.
2025-03-01 19:40:19,649 WETEXT INFO found existing fst: /usr/local/lib/python3.11/dist-packages/tn/zh_tn_tagger.fst
[+0000 20250301 19:40:19] [INFO] wetext-zh_normalizer | processor | found existing fst: /usr/local/lib/python3.11/dist-packages/tn/zh_tn_tagger.fst
2025-03-01 19:40:19,653 WETEXT INFO                     /usr/local/lib/python3.11/dist-packages/tn/zh_tn_verbalizer.fst
[+0000 20250301 19:40:19] [INFO] wetext-zh_normalizer | processor |                     /usr/local/lib/python3.11/dist-packages/tn/zh_tn_verbalizer.fst
2025-03-01 19:40:19,654 WETEXT INFO skip building fst for zh_normalizer ...
[+0000 20250301 19:40:19] [INFO] wetext-zh_normalizer | processor | skip building fst for zh_normalizer ...


In [16]:
# use force_redownload=True if the weights have been updated.
chat.load(source="huggingface")

rand_spk = chat.sample_random_speaker()
print(rand_spk) # save it for later timbre recovery

params_infer_code = ChatTTS.Chat.InferCodeParams(
    spk_emb = rand_spk, # add sampled speaker
    temperature = .1,   # using custom temperature
    top_P = 0.7,        # top P decode
    top_K = 20,         # top K decode
)

###################################
# For sentence level manual control.

# use oral_(0-9), laugh_(0-2), break_(0-7)
# to generate special token in text to synthesize.
params_refine_text = ChatTTS.Chat.RefineTextParams(
    prompt='[oral_2][break_6]',
)


[+0000 20250301 19:47:06] [INFO] ChatTTS | core | use device cuda:0
[+0000 20250301 19:47:06] [INFO] ChatTTS | core | vocos loaded.
[+0000 20250301 19:47:06] [INFO] ChatTTS | core | dvae loaded.
[+0000 20250301 19:47:07] [INFO] ChatTTS | core | embed loaded.
[+0000 20250301 19:47:09] [INFO] ChatTTS | core | gpt loaded.
[+0000 20250301 19:47:09] [INFO] ChatTTS | core | speaker loaded.
[+0000 20250301 19:47:09] [INFO] ChatTTS | core | decoder loaded.
[+0000 20250301 19:47:09] [INFO] ChatTTS | core | tokenizer loaded.
 NeMo-text-processing :: INFO     :: Skipping post-processing of after spending two weeks in hospital battling pneumonia , the speculation in the Vatican is whether his successor Pope Francis might do the same. "I think the only circumstances under which he (Francis) would consider resignation , as he has said , would be if he had a long-term degenerative or debilitating condition which prevented him from fully carrying out the papal ministry which prevented him from fully c

蘁淰敝欀椃跄糚搈膱宆謜氾熪秺泸愠蠺崤句悝素蘖腟叠猠媝蠙烼曼橫囝瀍觉師蝙証豯崸汮怱卻孜瀘婟笚摼戊蝆贑帹徫怳数炿薧虑尼柔蛗茦燡攠膘蓩篚丄婧奎竿嶽脆膸浞拵労獅虿螿匮壧梻慜惷洏傏袞碹褠摚甡箎獬曜絚豶悻罤槼槶蕤嫎谝胆蘰炐趞媇導詀晼肻妵蒫庩繖匚现愌翽蛻新稄缥膐划唩悫螩段责溺欑燾晐挎謰蓥蚫艊縤詺脭緭聿怩後樠蕒籔磏羠盜綖瀲專礽窰碉姰训籞蘿艜昘糿灝磩夊崫洔媍救姷燿咨犠芾脌櫫刃譶眈呲貣噚祺喢垻崠俴瀤濎襹皙坏綥潶枻嚔撘床仅伦蟹揱虭肔秐祹偑珝幐查藌緄搘誸彗萡斏仿貎耣塦珕超篻碥羁萲订塼澖楬笴譯疇愫秶犣惢槜縵珽栽葤碮茴燉樆之珼亷玃椑岫筛伞橇圖瘀漽甁桖譼褾坂掩厊嚳祟皗脭缽孋曐慈孁憁苕懽潸舻丶嫌姐猛愽暨怶蓎嬱懍斘趆渴啿狲朕訍倘赤咮瑑朓粈渁慅縃夓娦橒勦砯亪耥嬝儴洛灿竘詜唱诜榁痺灷匏捼營界碹羀檿控泠礳衵崈蔇丕脉曂嗅哂哐諫眱藋荀刳昉螩澘喖畎蠏坝纯卅賠襯瀂緺嘝櫺芯貐爞贛湏嶊倘谘摮泴憭唜烺溄俉喂畦睩缁谆宲旬摖蚦夯抐袓橒淺蟥墒椘暻趬腥詓崚圈嶛樨唜榩翇瑪腗嘳蓣穁座蓇煖課煶訸奈攚售犁赘謲埨樻底烴亱枹扦謔池掤艞藕拓濂澝命蔍脕囤珁抏繌耄纥恁濍喐舩聆嵖趜衉冎瞿匧垨调磆勡櫥灳珺婗炃焣嘁讂襑薏蕢觞婝巻綻赭珓蓜背娔瞅匦匤諽欄计珗仔絀浉啟蓥砛浴臊妭教憂癑涰湐讗戽吃弗壑贏漈揭罸潴患涳嵽稆涠戒彽变泫巪蚡猄寓碻珽潭屻蜄蒀屓礻褋曮僈旛儌藏燁映蟆托蟳褙喭岜嬃慴喴橹豰菘牘意朲招坼嵨怿惖慢抙蓕憆知妛榵绥戏螺爱覦奄怽聾呢櫯嵡仞炡岘搟煮瘕佬惟滊谀撯蓷荇詢楄葩熉埴藝苝罞嫠痬磓冈儽诺蚏膱璬溂兏劍乌晖焼憺廚欓侏亼舾宗统譽爓泊槥恩寋超烾詵徹嬫裢蕭得痛樐帳糭藃藳宋紁樨纉萶窑惰抠詂疦諥坩誳薲謃槔豢埳璭純哭荧救橐涁椛帑濰唕暅免濑塠婝綤岓疎浌蘍椒礆剎滘諘囄芸偢秦炖耂榋筑籽启斀燄聃尙箝诶沁殹烹棝瞱今荝撩洍琫崝瑶砃匋旫沲稶戄稲懰寭惲乽欰謭指峀苏赑爰螼肕疊贿紋婐簈桢濍哑婠嗿呓哔愼筬凯屫攊礑姌跆舲濶赫兢裥塼祹搕寗刍侀杬蜼痴烽畹姚脸舜温渃瀶喼蔢夔湾覓癣凒挙暧葩澆剓壌尙肘勴衶繌裇嶬肁玸妳磊徬旫蔹旑峐磍凞帘譴扣淑墏埻圣廑暖愺絕磧仜寻肑葶譟琏儔佼矄者膺廰墰帿浉柕掜晋娕燞赋庐磣債懞槓貋戕厡忄擫赯褩屭舀㴅


text:  63%|██████▎   | 241/384(max) [00:10, 23.24it/s]
[+0000 20250301 19:47:20] [INFO] ChatTTS | core | infer split 0~1
code:  66%|██████▌   | 1350/2048(max) [01:04, 20.91it/s]


In [7]:
# URL del artículo (solo traduce texto en inglés)
url = 'https://edition.cnn.com/2025/03/01/europe/pope-francis-health-resignation-speculation-vatican-intl/index.html'

In [45]:

import requests
from bs4 import BeautifulSoup
import ftfy
import unicodedata
import re

In [43]:
# Buscar el título del artículo
title = soup.find('h1').get_text()

# Eliminar signos de puntuación y convertir todo a minúsculas
title_cleaned = unicodedata.normalize('NFKD', title).encode('ascii', 'ignore').decode('ascii')
title_cleaned = re.sub(r'[^\w\s]', '', title_cleaned).lower().strip()
title_modified = title_cleaned.lower().replace(" ", "_")

# Mostrar el título
print("Título del artículo:", title_cleaned)

Título del artículo: would pope francis resign experts say itaos unlikely ai but the pontiff often springs surprises


In [23]:
# Obtenemos el texto de la noticia

# Realizar la solicitud HTTP
response = requests.get(url)

# Verificar que la solicitud fue exitosa
if response.status_code == 200:
    # Analizar el contenido HTML
    soup = BeautifulSoup(response.content, 'html.parser')

    # Encontrar el contenedor principal del artículo
    article_body = soup.find('div', class_='article__content')

    # Extraer el texto del artículo
    if article_body:
        paragraphs = article_body.find_all('p')
        article_text = '\n'.join([p.get_text() for p in paragraphs])
        fix_text = ftfy.fix_text(article_text)
        print(fix_text)
    else:
        print('No se pudo encontrar el contenido del artículo.')
else:
    print(f'Error al acceder a la página: {response.status_code}')


            When Benedict XVI became the first pope to resign in 600 years it sent shockwaves through the Catholic Church. Now, after spending two weeks in hospital battling pneumonia, the speculation in the Vatican is whether his successor Pope Francis might do the same.
    

            "I think the only circumstances under which he (Francis) would consider resignation, as he has said, would be if he had a long-term degenerative or debilitating condition which prevented him from fully carrying out the papal ministry," Austen Ivereigh, a papal biographer, told CNN.
    

            The pope was put on a breathing machine on Friday, after suffering a sudden episode of respiratory difficulty, the Vatican said. The episode was complicated by vomiting, some of which the pope aspirated, it added. A Vatican source said on Friday the next 24-48 hours will determine whether the pope's general condition has deteriorated. On Saturday morning, Francis was said to be resting, after what the Va

In [14]:
from transformers import pipeline

# Cargar el modelo de resumen
summarizer = pipeline("summarization", model="allenai/led-large-16384-arxiv")


# Generar el resumen
summary = summarizer(fix_text, max_length=300, min_length=100, do_sample=False)

# Mostrar el resumen
summary_text = summary[0]['summary_text']
print(summary_text)

Device set to use cuda:0


 after spending two weeks in hospital battling pneumonia , the speculation in the Vatican is whether his successor Pope Francis might do the same. "I think the only circumstances under which he (Francis) would consider resignation , as he has said , would be if he had a long-term degenerative or debilitating condition which prevented him from fully carrying out the papal ministry which prevented him from fully carrying out the papal ministry , as he has said , would be if he had a long-term degenerative or debilitating condition which prevented him from fully carrying out the papal ministry which prevented him from fully carrying out the papal ministry . " 


In [17]:
# Traduce el resumen de la noticia a audio
texts = [summary_text,]


wavs = chat.infer(
    texts,
    params_refine_text=params_refine_text,
    params_infer_code=params_infer_code,
)

 NeMo-text-processing :: INFO     :: Skipping post-processing of after spending two weeks in hospital battling pneumonia , the speculation in the Vatican is whether his successor Pope Francis might do the same. "I think the only circumstances under which he (Francis) would consider resignation , as he has said , would be if he had a long-term degenerative or debilitating condition which prevented him from fully carrying out the papal ministry which prevented him from fully carrying out the papal ministry , as he has said , would be if he had a long-term degenerative or debilitating condition which prevented him from fully carrying out the papal ministry which prevented him from fully carrying out the papal ministry. " for '.'
[+0000 20250301 19:49:09] [INFO] NeMo-text-processing | data_loader_utils | Skipping post-processing of after spending two weeks in hospital battling pneumonia , the speculation in the Vatican is whether his successor Pope Francis might do the same. "I think the o

In [18]:
# El audio que queremos exportar

Audio(wavs[0], rate=24_000, autoplay=True)

In [44]:
# Exportamos el audio con el titulo de la noticia
waveform = torch.from_numpy(wavs[0])
if waveform.dim() == 1:
    waveform = waveform.unsqueeze(0)

# Guardar el archivo de audio
torchaudio.save(title_modified + ".wav", waveform, 24000)